In [1]:
import pandas as pd
df = pd.read_csv("10SecondStepPriceEvolution2.csv")
df["F10"] = df["F10"] - df["Cur"]
df["P10"] = df["P10"] - df["Cur"]
df["P20"] = df["P20"] - df["Cur"]
df["P30"] = df["P30"] - df["Cur"]
df["P40"] = df["P40"] - df["Cur"]
df["P50"] = df["P50"] - df["Cur"]
df["P60"] = df["P60"] - df["Cur"]
df.drop('Cur', axis=1, inplace=True)
df['F10'] = df['F10'].apply(lambda x: 1 if x >= 1 else -1 if x <= -1 else 0)

In [2]:
zeros = df[df['F10'] == 0]
ones  = df[df['F10'] == 1]
negas = df[df['F10'] == -1]
zeros = zeros.sample(frac=1).reset_index(drop=True)
zeros = zeros[:15000]
df = pd.concat([zeros, ones, negas])
df = df.sample(frac=1).reset_index(drop=True)

In [3]:
train_dataset = df.head(40000)
test_dataset = df.tail(df.shape[0] - 40000)

In [4]:
import os
import pandas as pd
from torch.utils.data import Dataset

class CustomPriceDataset(Dataset):
    def __init__(self, dataframe, labels, transform=None, target_transform=None):
        self.dataframe = dataframe
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = list(self.dataframe.iloc[idx])
        label = self.labels.iloc[idx]
        if self.transform:
            data = self.transform(data)
        if self.target_transform:
            label = self.target_transform(label)
        return data, label

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Lambda

trnlbl = train_dataset['F10'] + 1
trnset = train_dataset.drop('F10', axis=1)
trainPrice = CustomPriceDataset(trnset, trnlbl, torch.FloatTensor, Lambda(lambda y: torch.zeros(3, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)))
train_dataloader = DataLoader(trainPrice, batch_size=64)

tstlbl = test_dataset['F10'] + 1
tstset = test_dataset.drop('F10', axis=1)
testPrice = CustomPriceDataset(tstset, tstlbl, torch.FloatTensor)
test_dataloader = DataLoader(testPrice, batch_size=64)

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(6, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 3),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
learning_rate = 1e-2
passed = 0

In [9]:
epochs = 100
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=1e-2, dampening=1e-3)

In [10]:
print("Start")
for t in range(epochs):
    print(f"Epoch {t+1 + passed}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
passed = passed + epochs
print("Done!")

Start
Epoch 1
-------------------------------
loss: 1.101977  [   64/40000]
loss: 1.073623  [ 6464/40000]
loss: 1.060048  [12864/40000]
loss: 1.053258  [19264/40000]
loss: 1.071350  [25664/40000]
loss: 1.061059  [32064/40000]
loss: 1.076380  [38464/40000]
Test Error: 
 Accuracy: 42.0%, Avg loss: 1.071490 

Epoch 2
-------------------------------
loss: 1.112722  [   64/40000]
loss: 1.071458  [ 6464/40000]
loss: 1.045130  [12864/40000]
loss: 1.042927  [19264/40000]
loss: 1.064198  [25664/40000]
loss: 1.050069  [32064/40000]
loss: 1.071813  [38464/40000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 1.067199 

Epoch 3
-------------------------------
loss: 1.116194  [   64/40000]
loss: 1.074010  [ 6464/40000]
loss: 1.041106  [12864/40000]
loss: 1.040251  [19264/40000]
loss: 1.061234  [25664/40000]
loss: 1.043204  [32064/40000]
loss: 1.070564  [38464/40000]
Test Error: 
 Accuracy: 43.6%, Avg loss: 1.064891 

Epoch 4
-------------------------------
loss: 1.115977  [   64/40000]
loss: 1.074812  [ 

loss: 1.060377  [ 6464/40000]
loss: 1.036769  [12864/40000]
loss: 0.995795  [19264/40000]
loss: 1.057732  [25664/40000]
loss: 1.004605  [32064/40000]
loss: 1.029313  [38464/40000]
Test Error: 
 Accuracy: 46.6%, Avg loss: 1.035631 

Epoch 29
-------------------------------
loss: 1.104116  [   64/40000]
loss: 1.059618  [ 6464/40000]
loss: 1.037495  [12864/40000]
loss: 0.994217  [19264/40000]
loss: 1.056635  [25664/40000]
loss: 1.003960  [32064/40000]
loss: 1.028729  [38464/40000]
Test Error: 
 Accuracy: 46.9%, Avg loss: 1.035336 

Epoch 30
-------------------------------
loss: 1.103996  [   64/40000]
loss: 1.059138  [ 6464/40000]
loss: 1.037224  [12864/40000]
loss: 0.993220  [19264/40000]
loss: 1.056553  [25664/40000]
loss: 1.002808  [32064/40000]
loss: 1.027637  [38464/40000]
Test Error: 
 Accuracy: 47.1%, Avg loss: 1.034962 

Epoch 31
-------------------------------
loss: 1.104065  [   64/40000]
loss: 1.058846  [ 6464/40000]
loss: 1.036875  [12864/40000]
loss: 0.992391  [19264/40000]
l

loss: 1.037549  [12864/40000]
loss: 0.981745  [19264/40000]
loss: 1.037391  [25664/40000]
loss: 0.988265  [32064/40000]
loss: 1.016552  [38464/40000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 1.030097 

Epoch 56
-------------------------------
loss: 1.092604  [   64/40000]
loss: 1.052938  [ 6464/40000]
loss: 1.038769  [12864/40000]
loss: 0.981405  [19264/40000]
loss: 1.035251  [25664/40000]
loss: 0.988422  [32064/40000]
loss: 1.016902  [38464/40000]
Test Error: 
 Accuracy: 48.2%, Avg loss: 1.029773 

Epoch 57
-------------------------------
loss: 1.092434  [   64/40000]
loss: 1.052480  [ 6464/40000]
loss: 1.039585  [12864/40000]
loss: 0.982541  [19264/40000]
loss: 1.035444  [25664/40000]
loss: 0.987601  [32064/40000]
loss: 1.016609  [38464/40000]
Test Error: 
 Accuracy: 48.3%, Avg loss: 1.029902 

Epoch 58
-------------------------------
loss: 1.090444  [   64/40000]
loss: 1.051877  [ 6464/40000]
loss: 1.037334  [12864/40000]
loss: 0.982242  [19264/40000]
loss: 1.034108  [25664/40000]
l

loss: 0.975053  [19264/40000]
loss: 1.015294  [25664/40000]
loss: 0.973056  [32064/40000]
loss: 0.990105  [38464/40000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 1.024970 

Epoch 83
-------------------------------
loss: 1.071898  [   64/40000]
loss: 1.037309  [ 6464/40000]
loss: 1.020374  [12864/40000]
loss: 0.975759  [19264/40000]
loss: 1.016638  [25664/40000]
loss: 0.973583  [32064/40000]
loss: 0.987493  [38464/40000]
Test Error: 
 Accuracy: 47.3%, Avg loss: 1.024899 

Epoch 84
-------------------------------
loss: 1.071307  [   64/40000]
loss: 1.035891  [ 6464/40000]
loss: 1.021245  [12864/40000]
loss: 0.973751  [19264/40000]
loss: 1.014394  [25664/40000]
loss: 0.972234  [32064/40000]
loss: 0.985678  [38464/40000]
Test Error: 
 Accuracy: 47.2%, Avg loss: 1.024668 

Epoch 85
-------------------------------
loss: 1.071820  [   64/40000]
loss: 1.034457  [ 6464/40000]
loss: 1.019244  [12864/40000]
loss: 0.975492  [19264/40000]
loss: 1.013132  [25664/40000]
loss: 0.969952  [32064/40000]
l